<a href="https://colab.research.google.com/github/Lufec/Caso_Dostoy/blob/main/Dostoy_3classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
np.set_printoptions(precision = 5,edgeitems = 11, linewidth = 200, suppress = True)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_excel("/content/gdrive/My Drive/CIÊNCIA DA COMPUTAÇÃO/CNN/Dolstoy_Profile.xls").drop(columns = 'ClienteNo')

Separar target em labels

In [ ]:
df.PrimeiraCompra.describe()

count    2840.000000
mean       53.112063
std        26.237098
min         0.000000
25%        32.900000
50%        51.010000
75%        71.360000
max       137.780000
Name: PrimeiraCompra, dtype: float64

In [ ]:
df.loc[df.PrimeiraCompra < 42, "PrimeiraCompra"] = 0
df.loc[(df.PrimeiraCompra >= 42) & (df.PrimeiraCompra < 66), "PrimeiraCompra"] = 1
df.loc[(df.PrimeiraCompra >= 66), "PrimeiraCompra"] = 2

In [ ]:
df.PrimeiraCompra.value_counts()

0.0    1064
1.0     899
2.0     877
Name: PrimeiraCompra, dtype: int64

In [ ]:
df_caso2 = df[df['PrimeiraCompra'] == 2]
df_caso1 = df[df['PrimeiraCompra'] == 1]
df_caso0 = df[df['PrimeiraCompra'] == 0]

Separar treino balanceadamente (819 casos "1" e 819 casos "0"). Teste fica com o resto (998 casos "1" e 204 casos "0")

In [ ]:
df2_train = df_caso2.head(701)
df1_train = df_caso1.head(701)
df0_train = df_caso0.head(701)
df2_test  = df_caso2.tail(176)
df1_test  = df_caso1.tail(198)
df0_test  = df_caso0.tail(363)
df_train = pd.concat([df2_train,df1_train,df0_train]).sample(frac = 1).reset_index(drop = True)
df_test = pd.concat([df2_test,df1_test,df0_test]).sample(frac = 1).reset_index(drop = True)
x_train = df_train.drop(columns = 'PrimeiraCompra')
y_train = df_train.PrimeiraCompra
x_test = df_test.drop(columns = 'PrimeiraCompra')
y_test = df_test.PrimeiraCompra

In [ ]:
x_train.head(5)

,Cidade,Idade,Sexo,EstadoCivil,NumeroFilhos,Escolaridade,Ocupacao,AreaAtuacao,NivelGerencial,TipoResidencia,CondicaoResidencia,ValorImovel,NoAutomoveis,Computador
0,4,34.96104,1,1,4,3,4,7,3,2,2,2,0,2
1,9,37.31681,2,3,0,3,3,6,3,2,2,2,1,2
2,5,34.32360,1,3,3,2,6,6,3,3,2,2,1,2
3,9,34.63310,2,3,0,3,3,4,3,2,2,3,1,2
4,4,30.45416,2,1,0,1,7,0,2,1,2,1,0,2


In [ ]:
featNum = ['Idade','NumeroFilhos','Escolaridade','NivelGerencial','TipoResidencia','CondicaoResidencia','ValorImovel','NoAutomoveis','Computador']
featCat = ['Cidade','EstadoCivil','Sexo','Ocupacao','AreaAtuacao']

In [ ]:
x_num = x_train[featNum].reset_index(drop = True)
x_cat = x_train[featCat].reset_index(drop = True)

Features Numéricas

In [ ]:
x_num = x_num.drop(columns = ['NumeroFilhos','Computador'])
#x_num.NivelGerencial = x_num.NivelGerencial - 1
#x_cat = x_cat.drop(columns = ['EstadoCivil'])

In [ ]:
x_num.head(5)

,Idade,Escolaridade,NivelGerencial,TipoResidencia,CondicaoResidencia,ValorImovel,NoAutomoveis
0,34.96104,3,3,2,2,2,0
1,37.31681,3,3,2,2,2,1
2,34.32360,2,3,3,2,2,1
3,34.63310,3,3,2,2,3,1
4,30.45416,1,2,1,2,1,0


Normalização

In [ ]:
x_num_mean = x_num.mean()
x_num_std = x_num.std()

In [ ]:
x_num_norm = (x_num-x_num_mean)/x_num_std

PCA

In [ ]:
correlation = np.corrcoef(x_num,rowvar = False)

In [ ]:
correlation

array([[1.     , 0.74384, 0.85049, 0.70861, 0.72452, 0.75209, 0.71321],
       [0.74384, 1.     , 0.72983, 0.67721, 0.65923, 0.6869 , 0.64701],
       [0.85049, 0.72983, 1.     , 0.72713, 0.77874, 0.82388, 0.79677],
       [0.70861, 0.67721, 0.72713, 1.     , 0.63575, 0.65642, 0.61319],
       [0.72452, 0.65923, 0.77874, 0.63575, 1.     , 0.70196, 0.66164],
       [0.75209, 0.6869 , 0.82388, 0.65642, 0.70196, 1.     , 0.69501],
       [0.71321, 0.64701, 0.79677, 0.61319, 0.66164, 0.69501, 1.     ]])

In [ ]:
eigvals,eigvect = np.linalg.eig(correlation)

In [ ]:
eigvals

array([5.29097, 0.11403, 0.22979, 0.41905, 0.28875, 0.32128, 0.33612])

In [ ]:
eigvect

array([[ 0.39364, -0.37253, -0.82525, -0.03806, -0.11239,  0.10557,  0.00173],
       [ 0.36708,  0.07908,  0.23083, -0.40707,  0.18967,  0.76794,  0.11961],
       [ 0.40909,  0.85717, -0.19649,  0.18436, -0.12183, -0.10018,  0.02084],
       [ 0.35749, -0.05565,  0.14311, -0.6861 ,  0.05397, -0.60236,  0.11022],
       [ 0.36883, -0.11803,  0.1508 ,  0.21135,  0.41743, -0.07968, -0.77591],
       [ 0.38066, -0.23924,  0.39003,  0.1975 , -0.77478,  0.0329 , -0.07325],
       [ 0.36635, -0.21439,  0.17811,  0.49442,  0.39889, -0.1372 ,  0.60474]])

In [ ]:
eigvals/sum(eigvals)

array([0.75585, 0.01629, 0.03283, 0.05986, 0.04125, 0.0459 , 0.04802])

In [ ]:
x_avet = eigvect[:,[0,2,5,6]]
x_avet

array([[ 0.39364, -0.82525,  0.10557,  0.00173],
       [ 0.36708,  0.23083,  0.76794,  0.11961],
       [ 0.40909, -0.19649, -0.10018,  0.02084],
       [ 0.35749,  0.14311, -0.60236,  0.11022],
       [ 0.36883,  0.1508 , -0.07968, -0.77591],
       [ 0.38066,  0.39003,  0.0329 , -0.07325],
       [ 0.36635,  0.17811, -0.1372 ,  0.60474]])

In [ ]:
x_num_dec = np.dot(x_num_norm,x_avet)
x_num_dec

array([[-0.90266,  0.07625,  0.78319, -0.35225],
       [-0.34697,  0.09625,  0.63733,  0.40556],
       [-0.32418,  0.32271, -1.12833,  0.44309],
       [-0.06206,  0.73257,  0.6419 ,  0.32901],
       [-3.09955, -0.51228,  0.03217, -0.71465],
       [-2.27144,  0.17157, -0.08154, -0.42048],
       [ 0.75736, -0.35539, -0.04576, -0.2453 ],
       [-1.41335,  0.79122, -0.21813,  0.26086],
       [-1.62115,  0.50969, -0.12749, -0.48195],
       [ 2.0055 , -0.04624, -0.40048, -0.54218],
       [-0.2115 , -0.18777,  0.67366,  0.40615],
       ...,
       [ 1.70172, -0.64299, -0.41011, -0.46572],
       [-0.57606, -0.60846,  0.87078, -0.35081],
       [ 3.54757,  0.68013,  0.21584,  0.28218],
       [-4.45186, -0.02313, -0.80244,  0.22407],
       [ 1.29513,  0.97522, -0.44247, -0.55877],
       [-0.24223, -0.12334,  0.66542,  0.40602],
       [ 0.68404, -0.25032, -0.28847,  0.58754],
       [-1.60287,  0.47136, -0.12259, -0.48187],
       [-2.12495, -0.24901,  0.9397 , -0.66162],
       [

In [ ]:
x_num_dec = pd.DataFrame(x_num_dec,columns = ['PC0','PC1','PC2','PC3'])

In [ ]:
x_num_dec

,PC0,PC1,PC2,PC3
0,-0.902655,0.076250,0.783189,-0.352247
1,-0.346973,0.096254,0.637332,0.405561
2,-0.324178,0.322708,-1.128334,0.443094
3,-0.062063,0.732570,0.641902,0.329014
4,-3.099551,-0.512281,0.032169,-0.714650
...,...,...,...,...
2098,0.684040,-0.250323,-0.288468,0.587542
2099,-1.602867,0.471362,-0.122591,-0.481870
2100,-2.124953,-0.249014,0.939697,-0.661620
2101,-0.362618,0.403297,-1.138644,0.442926


## Features Categoricas

In [ ]:
x_cat

,Cidade,EstadoCivil,Sexo,Ocupacao,AreaAtuacao
0,4,1,1,4,7
1,9,3,2,3,6
2,5,3,1,6,6
3,9,3,2,3,4
4,4,1,2,7,0
...,...,...,...,...,...
2098,4,1,1,4,0
2099,2,1,1,2,6
2100,10,1,2,6,8
2101,8,1,2,3,8


## One Hot Encode pra esses dois

In [ ]:
sexo = pd.get_dummies(x_cat['Sexo'], drop_first=False)
sexo.columns = ['Sexo_masculino', 'Sexo_feminino']

x_cat = pd.concat([x_cat,sexo], axis=1)
del x_cat['Sexo']

In [ ]:
estado_civil = pd.get_dummies(x_cat['EstadoCivil'], drop_first=False)
estado_civil.columns = ['EstadoCivil_solteiro', 'EstadoCivil_junto', 'EstadoCivil_casado', 'EstadoCivil_viuvo']
estado_civil

x_cat = pd.concat([x_cat,estado_civil], axis=1)
del x_cat['EstadoCivil']

In [ ]:
x_cat

,Cidade,Ocupacao,AreaAtuacao,Sexo_masculino,Sexo_feminino,EstadoCivil_solteiro,EstadoCivil_junto,EstadoCivil_casado,EstadoCivil_viuvo
0,4,4,7,1,0,1,0,0,0
1,9,3,6,0,1,0,0,1,0
2,5,6,6,1,0,0,0,1,0
3,9,3,4,0,1,0,0,1,0
4,4,7,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
2098,4,4,0,1,0,1,0,0,0
2099,2,2,6,1,0,1,0,0,0
2100,10,6,8,0,1,1,0,0,0
2101,8,3,8,0,1,1,0,0,0


## Feature Hashig para esses três

In [ ]:
from sklearn.feature_extraction import FeatureHasher

In [ ]:
x_cat[['Cidade','Ocupacao','AreaAtuacao']] = x_cat[['Cidade','Ocupacao','AreaAtuacao']].astype('string')

In [ ]:
x_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2103 entries, 0 to 2102
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Cidade                2103 non-null   string
 1   Ocupacao              2103 non-null   string
 2   AreaAtuacao           2103 non-null   string
 3   Sexo_masculino        2103 non-null   uint8 
 4   Sexo_feminino         2103 non-null   uint8 
 5   EstadoCivil_solteiro  2103 non-null   uint8 
 6   EstadoCivil_junto     2103 non-null   uint8 
 7   EstadoCivil_casado    2103 non-null   uint8 
 8   EstadoCivil_viuvo     2103 non-null   uint8 
dtypes: string(3), uint8(6)
memory usage: 61.7 KB


In [ ]:
fh1 = FeatureHasher(n_features=5, input_type='string')
hashed_features = fh1.fit_transform(x_cat['Cidade'])
hashed_features = hashed_features.toarray()
hashCidade = pd.DataFrame(data = hashed_features,columns = ['Cidade0','Cidade1','Cidade2','Cidade3','Cidade4'])
x_cat = pd.concat([x_cat,hashCidade],axis = 1).reset_index(drop = True)
del x_cat['Cidade']

In [ ]:
fh2 = FeatureHasher(n_features=4, input_type='string')
hashed_features = fh2.fit_transform(x_cat['Ocupacao'])
hashed_features = hashed_features.toarray()
hashOcupacao = pd.DataFrame(data = hashed_features,columns = ['Ocupacao0','Ocupacao1','Ocupacao2','Ocupacao3'])
x_cat = pd.concat([x_cat,hashOcupacao],axis = 1).reset_index(drop = True)
del x_cat['Ocupacao']

In [ ]:
fh3 = FeatureHasher(n_features=4, input_type='string')
hashed_features = fh3.fit_transform(x_cat['AreaAtuacao'])
hashed_features = hashed_features.toarray()
hashAreaAtuacao = pd.DataFrame(data = hashed_features,columns = ['AreaAtuacao0','AreaAtuacao1','AreaAtuacao2','AreaAtuacao3'])
x_cat = pd.concat([x_cat,hashAreaAtuacao],axis = 1).reset_index(drop = True)
del x_cat['AreaAtuacao']

In [ ]:
x_cat

,Sexo_masculino,Sexo_feminino,EstadoCivil_solteiro,EstadoCivil_junto,EstadoCivil_casado,EstadoCivil_viuvo,Cidade0,Cidade1,Cidade2,Cidade3,Cidade4,Ocupacao0,Ocupacao1,Ocupacao2,Ocupacao3,AreaAtuacao0,AreaAtuacao1,AreaAtuacao2,AreaAtuacao3
0,1,0,1,0,0,0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,1,0,0,1,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,1,0,0,1,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
4,0,1,1,0,0,0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,1,0,1,0,0,0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
2099,1,0,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2100,0,1,1,0,0,0,0.0,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
2101,0,1,1,0,0,0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0


In [ ]:
df_cat = pd.concat([x_cat,y_train],axis = 1)

In [ ]:
x_train_novo = pd.concat([x_num_dec,x_cat],axis =  1)

## Acertando o conjunto de treino..

In [ ]:
x_num_test = x_test[featNum].drop(columns = ['Computador','NumeroFilhos'])
#x_num.NivelGerencial = x_num.NivelGerencial - 1
x_cat_test = x_test[featCat]

In [ ]:
x_test_norm = (x_num_test-x_num_mean)/x_num_std

In [ ]:
x_test_dec = np.dot(x_test_norm,x_avet)

In [ ]:
x_test_dec = pd.DataFrame(x_test_dec,columns = ['PC0','PC1','PC2','PC3'])

In [ ]:
sexo = pd.get_dummies(x_cat_test['Sexo'], drop_first=False)
sexo.columns = ['Sexo_masculino', 'Sexo_feminino']

x_cat_test = pd.concat([x_cat_test,sexo], axis=1)
del x_cat_test['Sexo']  

In [ ]:
estado_civil = pd.get_dummies(x_cat_test['EstadoCivil'], drop_first=False)
estado_civil.columns = ['EstadoCivil_solteiro', 'EstadoCivil_junto', 'EstadoCivil_casado', 'EstadoCivil_viuvo']
estado_civil

x_cat_test = pd.concat([x_cat_test,estado_civil], axis=1)
del x_cat_test['EstadoCivil']

In [ ]:
x_cat_test[['Cidade','Ocupacao','AreaAtuacao']] = x_cat_test[['Cidade','Ocupacao','AreaAtuacao']].astype('string')

In [ ]:
hashed_features = fh1.transform(x_cat_test['Cidade'])
hashed_features = hashed_features.toarray()
hashCidade = pd.DataFrame(data = hashed_features,columns = ['Cidade0','Cidade1','Cidade2','Cidade3','Cidade4'])
x_cat_test = pd.concat([x_cat_test,hashCidade],axis = 1).reset_index(drop = True)
del x_cat_test['Cidade']

In [ ]:
hashed_features = fh2.transform(x_cat_test['Ocupacao'])
hashed_features = hashed_features.toarray()
hashOcupacao = pd.DataFrame(data = hashed_features,columns = ['Ocupacao0','Ocupacao1','Ocupacao2','Ocupacao3'])
x_cat_test = pd.concat([x_cat_test,hashOcupacao],axis = 1).reset_index(drop = True)
del x_cat_test['Ocupacao']

In [ ]:
hashed_features = fh3.transform(x_cat_test['AreaAtuacao'])
hashed_features = hashed_features.toarray()
hashAreaAtuacao = pd.DataFrame(data = hashed_features,columns = ['AreaAtuacao0','AreaAtuacao1','AreaAtuacao2','AreaAtuacao3'])
x_cat_test = pd.concat([x_cat_test,hashAreaAtuacao],axis = 1).reset_index(drop = True)
del x_cat_test['AreaAtuacao']

In [ ]:
x_test_novo = pd.concat([x_test_dec,x_cat_test],axis =  1)

## SVM e MLP


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
svm = SVC(C=1.0,
          kernel='rbf',        #kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’       
          gamma='auto' ,           # se gamma='scale'(padrão) é passado, então ele usa 1 / (n_features * X.var ()) como valor de gama,
                                   # se 'auto', usa 1 / n_features.
          degree = 3,              # grau do polinomio para kernel poly
          coef0 = 5,               # usa em poly e sigmoid
          tol=0.1,               # Tolerância para critério de parada.
          cache_size=2048,         # Especifique o tamanho do cache do kernel (em MB).
          verbose= False, 
          max_iter=-1,             # Limite rígido nas iterações no solver ou -1 para nenhum limite.
          )

svm.fit(x_train_novo, y_train.values.ravel())

print("Treino\n")
predictions = svm.predict(x_train_novo)
print(confusion_matrix(y_train,predictions))
print(classification_report(y_train,predictions))

print("\n Teste\n")
predictionsTestSvm = svm.predict(x_test_novo)
print(confusion_matrix(y_test,predictionsTestSvm))
print(classification_report(y_test,predictionsTestSvm))

Treino

[[526 168   7]
 [119 462 120]
 [  5 158 538]]
              precision    recall  f1-score   support

         0.0       0.81      0.75      0.78       701
         1.0       0.59      0.66      0.62       701
         2.0       0.81      0.77      0.79       701

    accuracy                           0.73      2103
   macro avg       0.73      0.73      0.73      2103
weighted avg       0.73      0.73      0.73      2103


 Teste

[[293  68   2]
 [ 37 134  27]
 [  3  45 128]]
              precision    recall  f1-score   support

         0.0       0.88      0.81      0.84       363
         1.0       0.54      0.68      0.60       198
         2.0       0.82      0.73      0.77       176

    accuracy                           0.75       737
   macro avg       0.75      0.74      0.74       737
weighted avg       0.77      0.75      0.76       737



Acertando o conjunto de treino para executar a predição

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp1 = MLPClassifier(solver = 'adam',    #adam, sgd, lbfgs
                     activation ='relu',       #identity, logistic, tanh, relu
                     hidden_layer_sizes=(8,8,8), 
                     alpha = 0.1,
                     learning_rate = 'constant',    #constant, invscaling, adaptive . só para sgd
                     learning_rate_init = 0.01,    #so para sgd
                     power_t = 0.1,                 #so para sgd, usando invscaling
                     momentum = 0.9,                #so para sgd
                     nesterovs_momentum = True,      #so para sgd e se momentum > 0
                     beta_1 = 0.9,                  #so para adam
                     beta_2 = 0.99,                  #so para adam
                     epsilon = 1e-8,                #so para adam
                     tol = 1e-3,
                     max_iter=1000,
                     verbose = False, 
                     early_stopping= True,
                     validation_fraction= 0.2,      #so se early stopping = true
                     shuffle = True, 
                     n_iter_no_change = 50)



mlp1.fit(x_train_novo, 
         y_train.values.ravel())
predictions = mlp1.predict(x_train_novo)
print("Treino\n")
print(confusion_matrix(y_train,predictions))
print(classification_report(y_train,predictions))
print("\n Teste \n")
predictionsTestMlp = mlp1.predict(x_test_novo)
print(confusion_matrix(y_test,predictionsTestMlp))
print(classification_report(y_test,predictionsTestMlp))

Treino

[[558 137   6]
 [145 448 108]
 [  8 160 533]]
              precision    recall  f1-score   support

         0.0       0.78      0.80      0.79       701
         1.0       0.60      0.64      0.62       701
         2.0       0.82      0.76      0.79       701

    accuracy                           0.73      2103
   macro avg       0.74      0.73      0.73      2103
weighted avg       0.74      0.73      0.73      2103


 Teste 

[[301  60   2]
 [ 45 129  24]
 [  4  44 128]]
              precision    recall  f1-score   support

         0.0       0.86      0.83      0.84       363
         1.0       0.55      0.65      0.60       198
         2.0       0.83      0.73      0.78       176

    accuracy                           0.76       737
   macro avg       0.75      0.74      0.74       737
weighted avg       0.77      0.76      0.76       737



## Contrapondo o Prospect

In [ ]:
df2 = pd.read_excel("/content/gdrive/My Drive/CIÊNCIA DA COMPUTAÇÃO/CNN/Profile_Prospect.xls").drop(columns = ['ClienteNo'])

In [ ]:
df2['Lucro'] = df2['PrimeiraCompra'].copy() - 42

In [ ]:
df2.loc[df2.PrimeiraCompra < 42, "PrimeiraCompra"] = 0
df2.loc[df2.PrimeiraCompra >= 66, "PrimeiraCompra"] = 2
df2.loc[df2.PrimeiraCompra >= 42, "PrimeiraCompra"] = 1

In [ ]:
x_prof = df2.drop(columns = 'PrimeiraCompra')
y_prof = df2['PrimeiraCompra']

In [ ]:
featNum = ['Idade','NumeroFilhos','Escolaridade','NivelGerencial','TipoResidencia','CondicaoResidencia','ValorImovel','NoAutomoveis','Computador']
featCat = ['Cidade','EstadoCivil','Sexo','Ocupacao','AreaAtuacao']

In [ ]:
x_num_prof = x_prof[featNum].reset_index(drop = True)
#x_num.NivelGerencial = x_num.NivelGerencial - 1
x_cat_prof = x_prof[featCat].reset_index(drop = True)

In [ ]:
x_num_prof = x_prof[featNum].drop(columns = ['Computador','NumeroFilhos'])
x_cat_prof = x_prof[featCat]

In [ ]:
x_prof_norm = (x_num_prof-x_num_mean)/x_num_std

In [ ]:
x_prof_dec = np.dot(x_prof_norm,x_avet)

In [ ]:
x_prof_dec = pd.DataFrame(x_prof_dec,columns = ['PC0','PC1','PC2','PC3'])

In [ ]:
sexo = pd.get_dummies(x_cat_prof['Sexo'], drop_first=False)
sexo.columns = ['Sexo_masculino', 'Sexo_feminino']

x_cat_prof = pd.concat([x_cat_prof,sexo], axis=1)
del x_cat_prof['Sexo']  

In [ ]:
estado_civil = pd.get_dummies(x_cat_prof['EstadoCivil'], drop_first=False)
estado_civil.columns = ['EstadoCivil_solteiro', 'EstadoCivil_junto', 'EstadoCivil_casado', 'EstadoCivil_viuvo']
estado_civil

x_cat_prof = pd.concat([x_cat_prof,estado_civil], axis=1)
del x_cat_prof['EstadoCivil']

In [ ]:
x_cat_prof[['Cidade','Ocupacao','AreaAtuacao']] = x_cat_prof[['Cidade','Ocupacao','AreaAtuacao']].astype('string')

In [ ]:
hashed_features = fh1.transform(x_cat_prof['Cidade'])
hashed_features = hashed_features.toarray()
hashCidade = pd.DataFrame(data = hashed_features,columns = ['Cidade0','Cidade1','Cidade2','Cidade3','Cidade4'])
x_cat_prof = pd.concat([x_cat_prof,hashCidade],axis = 1).reset_index(drop = True)
del x_cat_prof['Cidade']

In [ ]:
hashed_features = fh2.transform(x_cat_prof['Ocupacao'])
hashed_features = hashed_features.toarray()
hashOcupacao = pd.DataFrame(data = hashed_features,columns = ['Ocupacao0','Ocupacao1','Ocupacao2','Ocupacao3'])
x_cat_prof = pd.concat([x_cat_prof,hashOcupacao],axis = 1).reset_index(drop = True)
del x_cat_prof['Ocupacao']

In [ ]:
hashed_features = fh3.transform(x_cat_prof['AreaAtuacao'])
hashed_features = hashed_features.toarray()
hashAreaAtuacao = pd.DataFrame(data = hashed_features,columns = ['AreaAtuacao0','AreaAtuacao1','AreaAtuacao2','AreaAtuacao3'])
x_cat_prof = pd.concat([x_cat_prof,hashAreaAtuacao],axis = 1).reset_index(drop = True)
del x_cat_prof['AreaAtuacao']

In [ ]:
x_prof_novo = pd.concat([x_prof_dec,x_cat_prof],axis =  1)

In [ ]:
predictionsProfSVM = svm.predict(x_prof_novo)
print(confusion_matrix(y_prof,predictionsProfSVM))
print(classification_report(y_prof,predictionsProfSVM))

[[669 242   9]
 [160 475 143]
 [ 13 196 549]]
              precision    recall  f1-score   support

         0.0       0.79      0.73      0.76       920
         1.0       0.52      0.61      0.56       778
         2.0       0.78      0.72      0.75       758

    accuracy                           0.69      2456
   macro avg       0.70      0.69      0.69      2456
weighted avg       0.70      0.69      0.69      2456



In [ ]:
predictionsProfMLP = mlp1.predict(x_prof_novo)
print(confusion_matrix(y_prof,predictionsProfMLP))
print(classification_report(y_prof,predictionsProfMLP))

[[699 211  10]
 [206 439 133]
 [ 21 210 527]]
              precision    recall  f1-score   support

         0.0       0.75      0.76      0.76       920
         1.0       0.51      0.56      0.54       778
         2.0       0.79      0.70      0.74       758

    accuracy                           0.68      2456
   macro avg       0.68      0.67      0.68      2456
weighted avg       0.69      0.68      0.68      2456



In [ ]:
df2['Lucro'].sum()

27290.28

In [ ]:
df2.loc[(predictionsProfSVM != 0), 'Lucro'].sum()

37671.47

In [ ]:
df2.loc[(predictionsProfMLP != 0), 'Lucro'].sum()

37261.119999999995